In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import paper_classes_2 as pc
import Post_processing as pp
import itertools
import os
import import_ipynb
import comm_as_today_run as comm
import pickle
from functools import wraps
import time
import multiprocessing as mp

importing Jupyter notebook from comm_as_today_run.ipynb


# First let's load the Swiss community with 100 houses from the CREM dataset

if it is the first time you run this script set first_time to True (to select randomly the demand profiles and the PV sizes), otherwise set it to False

In [2]:
def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print ("Total time running %s: %s seconds" %
               (function.__name__, str(t1-t0))
               )
        return result
    return function_timer

In [3]:
def sell_prob(SOC,price,Capacity,prob_mat,I):
    '''
    Description
    -----------
    This function returns the selling probability based on the SOC, price, capacity and case. As well as the results of the survey which are stored as .txt
    Parameters
    ------
    SOC: float; Battery state of charge at time t
    price: float; Electricity price at time t
    Capacity: float; Battery capacity
    case: string; either PV or batt from where the user is willing to sale
    path: string; path where the .txt are found
    ------
    out: int; Either 1 or 0 if sell or not
    TODO
    ------
    '''
    
    if SOC==0:
        out=0
    else:
        ind=find_interval(SOC,I*Capacity)
        if ind==-1:
            prob=prob_mat.loc[1,str(price)]
        else:
            prob=prob_mat.loc[I[ind],str(price)]
        out=np.random.choice(2,1,p=[1-prob,prob]) 
    return out

In [4]:

def find_interval_PQ(x, partition):
    '''
    Description
    -----------
    find_interval at which x belongs inside partition. Returns the index i.

    Parameters
    ------
    x: float; numerical value
    partition: array; sequence of numerical values
    Returns
    ------
    i: index; index for which applies
    partition[i] < x < partition[i+1], if such an index exists.
    -1 otherwise
    TODO
    ------
    '''
    
    for i in range(0, len(partition)):
        #print(partition)
        if x<partition[1]:
            return 1
        elif x < partition[i]:
            return i-1
        
    return -1

In [5]:
def find_interval(x, partition):
    '''
    Description
    -----------
    find_interval at which x belongs inside partition. Returns the index i.

    Parameters
    ------
    x: float; numerical value
    partition: array; sequence of numerical values
    Returns
    ------
    i: index; index for which applies
    partition[i] < x < partition[i+1], if such an index exists.
    -1 otherwise
    TODO
    ------
    '''

    for i in range(0, len(partition)):
        if x < partition[i]:
            return i-1
    return -1



In [6]:
def get_surplus_hour(res_pv):
    '''
    Description
    -----------
    Get the first surplus hour
    Parameters
    -----------
    res_pv (pd.Series): Series of residual pv (surplus)
    Returns
    -----------
    sp_hour (list): list of daily hour when there is surplus
    '''
    sp_hour=[]
    if int(res_pv.shape[0]/365)==24:
        aux_res_pv=np.array(res_pv).reshape((365,24))
    else:
        print('TODO')
        
        
    for day in range(0,365):
        sp_hour.append(np.argmax(aux_res_pv.reshape((365,24))[day]>0))
    return sp_hour

In [7]:
def dispatch_probs(pv, demand, param, return_series=False):
    """ 
    Description
    -----------
    Self consumption maximization pv + battery dispatch algorithm.
    The dispatch of the storage capacity is performed in such a way to maximize self-consumption:
    the battery is charged when the PV power is higher than the load and as long as it is not fully charged.
    It is discharged as soon as the PV power is lower than the load and as long as it is not fully discharged.
    
    Modified by alefunxo (01.07.2019) to include other parameters and forbid charge and discharge in the same timeslot
    and inverter and battery losses. Moreover, it includes probabilities to discharge X kW from the battery to the community
    or to avoid charging X kW from the PV to inject it to the community.
    
    Parameters
    -----------
    pv (pd.Series): Vector of PV generation, in kW DC (i.e. before the inverter)
    demand (pd.Series): Vector of household consumption, kW
    param (dict): Dictionary with the simulation parameters:
            timestep (float): Simulation time step (in hours)
            BatteryCapacity: Available battery capacity (i.e. only the the available DOD), kWh
            BatteryEfficiency: Battery round-trip efficiency, -
            InverterEfficiency: Inverter efficiency, -
            MaxPower: Maximum battery charging or discharging powers (assumed to be equal), kW
    return_series(bool): if True then the return will be a dictionary of series. Otherwise it will be a dictionary of ndarrays.
                    It is reccommended to return ndarrays if speed is an issue (e.g. for batch runs).
    Returns
    -----------
    dict: Dictionary of Time series

        
    """
    
        
    
    kW_dis= param['kW_dis']
    bat_size_e_adj = param['BatteryCapacity']
    bat_size_p_adj = param['MaxPower']
    n_bat = param['BatteryEfficiency']
    n_inv = param['InverterEfficiency']
    timestep = param['timestep']
    # We work with np.ndarrays as they are much faster than pd.Series
    Nsteps = len(pv)
    LevelOfCharge = np.zeros(Nsteps)
    pv2store = np.zeros(Nsteps)
    inv2grid = np.zeros(Nsteps)
    store2inv = np.zeros(Nsteps)
    store2load = np.zeros(Nsteps)
    grid2store = np.zeros(Nsteps) # TODO Always zero for now.
    store2grid = np.zeros(Nsteps)
    batt_losses = np.zeros(Nsteps)
    flag_sp_p2p=np.zeros(Nsteps)
    flag_no_sp_p2p=np.zeros(Nsteps)
    prices = np.array(param['df_prices']).flatten()
    #prices=np.ones(Nsteps)*.07
    #Load served by PV
    pv2inv = np.minimum(pv, demand / n_inv)  # DC direct self-consumption

    #Residual load
    res_load = (demand - pv2inv * n_inv)  # AC
    inv2load = pv2inv * n_inv  # AC

    #Excess PV
    res_pv = np.maximum(pv - demand/n_inv, 0)  # DC
    #PV to storage after eff losses
    pv2inv = pv2inv.values
    
    #first timestep = 0
    LevelOfCharge[0] = 0  # bat_size_e_adj / 2  # DC
    ######################################
    
    if (bat_size_e_adj!=0)&(pv.sum()>0):
        sp_hour=get_surplus_hour(res_pv)
        for i in range(1,Nsteps):
            ######################################
            if sp_hour[int(np.floor((i)/24))]==0:#no surplus at all
                flag_sp=3
                prob_mat=param['prob_mat3']
            elif i%24<sp_hour[int(np.floor((i)/24))]-2:#2h bf surplus
                flag_sp=1
                prob_mat=param['prob_mat1']
            elif i%24>sp_hour[int(np.floor((i)/24))]-14+24: #14h bf surplus
                flag_sp=3
                prob_mat=param['prob_mat3']
            else:#while sp
                flag_sp=2
                prob_mat=param['prob_mat2']
            ######################################
            #PV to storage
            if (sell_prob(LevelOfCharge[i-1],prices[i],bat_size_e_adj,prob_mat,param['interval_SOC'])&(flag_sp==2)):
                flag_no_sp_p2p[i]=1
                aux=res_pv[i]-min(kW_dis/n_inv/timestep,res_pv[i])
                pv2inv[i]=pv2inv[i]+aux
                res_pv[i]=res_pv[i]-aux
                inv2grid[i]=aux#DC

            if LevelOfCharge[i-1] >= bat_size_e_adj:  # if battery is full
                    pv2store[i] = 0
            else: #if battery is not full
                if LevelOfCharge[i-1] + res_pv[i] * timestep > bat_size_e_adj:  # if battery will be full after putting excess
                    pv2store[i] = min((bat_size_e_adj - LevelOfCharge[i-1]) / timestep, bat_size_p_adj)
                else:
                    pv2store[i] = min(res_pv[i], bat_size_p_adj)

            #Storage to load or grid
            ######################################
            batt_losses[i]=pv2store[i]*(1-n_bat)
            if pv2store[i]==0:#not charging
                store2load[i] = min(bat_size_p_adj,  # DC
                               res_load[i] / n_inv,
                               LevelOfCharge[i-1] / timestep)
                if (sell_prob(LevelOfCharge[i-1]-store2load[i],prices[i],bat_size_e_adj,prob_mat,param['interval_SOC'])&(flag_sp!=2)):
                    flag_sp_p2p[i]=1
                    store2grid[i] = min(bat_size_p_adj-store2load[i]/timestep,  # DC
                                       kW_dis / n_inv/timestep/n_bat,
                                       (LevelOfCharge[i-1]-store2load[i]) / timestep)
                    store2inv[i] = (store2load[i]+store2grid[i])
                else:
                    store2inv[i]=store2load[i]
            ######################################a

            #SOC
            LevelOfCharge[i] = min(LevelOfCharge[i-1] - batt_losses[i]-(store2inv[i] - pv2store[i] - grid2store[i]) * timestep,  # DC
                                   bat_size_e_adj)

    store2grid=store2grid # AC
    pv2inv = pv2inv + res_pv - pv2store
    inv2load = inv2load + store2load * n_inv  # AC
    inv2grid = (inv2grid+res_pv - pv2store+store2grid) * n_inv  # AC
    grid2load = demand - inv2load  # AC
    inv_losses=(pv2inv+store2inv)*(1-n_inv)

    #MaxDischarge = np.minimum(LevelOfCharge[i-1]*BatteryEfficiency/timestep,MaxPower)


    #Potential Grid to storage  # TODO: not an option for now in this strategy
    # GridPurchase = False

    out = {'pv2inv': pv2inv,
            'res_pv': res_pv,
            'pv2store': pv2store,
            'inv2load': inv2load,
            'grid2load': grid2load,
            'store2inv': store2inv,
            'LevelOfCharge': LevelOfCharge,
            'inv2grid': inv2grid,
            'store2grid':store2grid,
            'store2load':store2load,
            'batt_losses':batt_losses,
            'inv_losses':inv_losses,
            'flag_no_sp_p2p':flag_no_sp_p2p,
            'flag_sp_p2p':flag_sp_p2p
            # 'grid2store': grid2store
            }
    if not return_series:
        out_pd = {}
        for k, v in out.items():  # Create dictionary of pandas series with same index as the input pv
            out_pd[k] = pd.Series(v, index=pv.index)
        out = out_pd
    return out

In [8]:

def price_probability(df_CH,selection,inputs,pv_penetration,batt_penetration):
    '''
    Description
    -----------
    Define the price structure according to the inputs (how the community is created), the choice of the probability, 
    whether it is for the 50% or other probability according to the psychology data, the amount of kWh that will be 
    discharged every time a decision is made (1 or 2 kWh are the "normal" options) and the case, whether it is the
    price created from the probability of selling from PV or the battery in the morning or the evening.
    
    Parameters
    -----------
    inputs:        dict; Dictionary that includes prob_choice (choice of the probability to create the prices 50% or other),
                    prices, kWh_dis (amount of energy to be discharged at every time a decision is made),  
                    case (included in ['PV', 'Batt'] the price may be created from the probability of selling from PV or the battery

    Return
    -----------
    inputs: dict; Input dictionary updated with prices

    TODO
    ------

    '''
    list_names=[str(name) for name in selection.name]
    # Get the total demand
    sum_demand=df_CH.loc[:,list_names].sum(axis=1)
    # Get total PV generation
    aux=pd.DataFrame()
    sum_PV=pd.DataFrame()
    for i in range(len(selection)):
        aux=df_CH.E_PV*selection.loc[:,['PV_size','sub_'+str(pv_penetration)+'_'+str(batt_penetration)]].product(axis=1)[i]
        if i==0:
            sum_PV=aux
        else:
            sum_PV=aux+sum_PV

    net_demand=sum_demand-sum_PV
    prob_mat_PV=pd.read_csv('Switzerland/Input/P_selling_by_price_and_autarky2.txt',sep='\t',index_col=[0])
    prob_mat_Batt=pd.read_csv('Switzerland/Input/P_selling_by_price_and_autarky_2.txt',sep='\t',index_col=[0])
    sp_hour=get_surplus_hour(sum_PV-sum_demand)
    if len(inputs['prob_choice'])==1:
        q_supply_midday=int(np.floor(inputs['PV_penetration']*inputs['Batt_penetration']*inputs['community_size']))*prob_mat_PV.loc[inputs['prob_choice'][0],:]*inputs['kW_dis']
        q_supply_other=int(np.floor(inputs['PV_penetration']*inputs['Batt_penetration']*inputs['community_size']))*prob_mat_Batt.loc[inputs['prob_choice'][0],:]*inputs['kW_dis']
        df_prices=pd.DataFrame(index=net_demand.index)
        df_prices.loc[:,'prices']=net_demand.apply(lambda x:inputs['prices'][find_interval_PQ(x,q_supply_midday)])
        df_prices.loc[(df_prices.index.hour<sp_hour[int(np.floor((i)/24))]-2)|(df_prices.index.hour>sp_hour[int(np.floor((i)/24))]-14+24),'prices']=net_demand[(net_demand.index.hour<sp_hour[int(np.floor((i)/24))]-2)|(net_demand.index.hour>sp_hour[int(np.floor((i)/24))]-14+24)].apply(lambda x:inputs['prices'][find_interval_PQ(x,q_supply_other)])
        inputs.update({'df_prices':df_prices})
        return inputs
    elif len(inputs['prob_choice'])==3:
        q_supply_morning=int(np.floor(inputs['PV_penetration']*inputs['Batt_penetration']*inputs['community_size']))*prob_mat_Batt.loc[inputs['prob_choice'][0],:]*inputs['kW_dis']
        q_supply_midday=int(np.floor(inputs['PV_penetration']*inputs['Batt_penetration']*inputs['community_size']))*prob_mat_PV.loc[inputs['prob_choice'][1],:]*inputs['kW_dis']
        q_supply_evening=int(np.floor(inputs['PV_penetration']*inputs['Batt_penetration']*inputs['community_size']))*prob_mat_Batt.loc[inputs['prob_choice'][2],:]*inputs['kW_dis']
        df_prices=pd.DataFrame(index=net_demand.index)
        df_prices.loc[:,'prices']=net_demand.apply(lambda x:inputs['prices'][find_interval_PQ(x,q_supply_midday)])
        df_prices.loc[(df_prices.index.hour<sp_hour[int(np.floor((i)/24))]-2),'prices']=net_demand[net_demand.index.hour<sp_hour[int(np.floor((i)/24))]-2].apply(lambda x:inputs['prices'][find_interval_PQ(x,q_supply_morning)])
        df_prices.loc[df_prices.index.hour>sp_hour[int(np.floor((i)/24))]-14+24,'prices']=net_demand[net_demand.index.hour>sp_hour[int(np.floor((i)/24))]-14+24].apply(lambda x:inputs['prices'][find_interval_PQ(x,q_supply_evening)])

        inputs.update({'df_prices':df_prices})
        return inputs
    else:
        print('Warning: this function only takes either one choice of probability for the whole day or three separated for morning, midday and evening')
        return

In [9]:
@fn_timer
def run_comm_p2p_test(inputs,pv_penetration,batt_penetration):
    '''
    Description
    -----------
    Core function of the P2P community. Does the dispatch of electricity for every
    household taking into account whether the houses have or not PV and battery for the selected 
    PV and battery penetration parameters as well as the probabilities to export to the grid.
    Recalculates the outputs for the whole community taking the surplus generated inside the 
    community (new input as PV generation for the dispatching) and the residual load after 
    self-consumption (new input as demand for the dispatching). Saves the results in pickle objects 
    to calculate the bill after.    
    Parameters
    -----------
    inputs (array): [use_cluster1_probabilities,save]
                    use_cluster1_probabilities (Boolean): Whether to use cluster 1 probs or not (cluster 2)
                    save (Boolean): Whether to save or not the community_p2p object (dict)
    pv_penetration (int): % PV penetration
    batt_penetration (int): % battery penetration
    Returns
    -----------
    None
    Comments
    -----------
    if prob_choice len is one then the prices are defined using 50% of battery SOC all day round. 
    If prob_choice is 3 then the three choices are used to create the supply at morning, midday and evening.
    '''
    community_size=100
    timestep=1 # in hours
    use_cluster1_probabilities=inputs[0]#cluster1 is Price+Autarky cluster2 Mostly Autarkie
    save=inputs[1]
    list_pv_penetration=[100,75,50,25]# %
    list_batt_penetration=[100,75,50,25]# %
    path=os.getcwd()+'/Switzerland/'
    #define the matrix of probs
    # while surplus
    
    if use_cluster1_probabilities:
        prob_mat2=pd.read_csv(path+'Input/surplus_cluster1_p_selling.csv',sep=';',engine='python',index_col=[0])
        #2h before surplus
        prob_mat1=pd.read_csv(path+'Input/nosurplus_cluster1_p_selling.csv',sep=';',engine='python',index_col=[0])
        #14h before surplus
        prob_mat3=pd.read_csv(path+'Input/nosurplus_cluster1_p_selling.csv',sep=';',engine='python',index_col=[0])
    else:
        prob_mat2=pd.read_csv(path+'Input/surplus_cluster2_p_selling.csv',sep=';',engine='python',index_col=[0])
        #2h before surplus
        prob_mat1=pd.read_csv(path+'Input/nosurplus_cluster2_p_selling.csv',sep=';',engine='python',index_col=[0])
        #14h before surplus
        prob_mat3=pd.read_csv(path+'Input/nosurplus_cluster2_p_selling.csv',sep=';',engine='python',index_col=[0])
    

    print('\n########################################')
    print('PV penetration {}'.format(pv_penetration))
    print('Battery penetration {}'.format(batt_penetration))
    out_comm_final=comm.load_obj('community_as_today'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
    param_tech=out_comm_final['param_tech']

    df_CH=out_comm_final['df']
    selection=out_comm_final['selection']

    param_tech.update({'kW_dis':1,'PV_penetration':pv_penetration/100,'Batt_penetration':batt_penetration/100,
                       'prob_choice':[0.35,0.5,.95],
                'prices':np.array([0, 0.07,.10,.13,.16,.19,.22,.25,.28]),'community_size':community_size,
                'prob_mat1':prob_mat1,'prob_mat2':prob_mat2,'prob_mat3':prob_mat3,
                       'use_cluster1_probabilities':use_cluster1_probabilities,
                'interval_SOC':np.array([0,0.05,0.2,0.35,0.5,0.65,0.8,0.95,1.0001]),'path':path})
    price_probability(df_CH,selection,param_tech,pv_penetration,batt_penetration)
    
    # We set the battery capacity to zero in some cases to get the direct SC only
    # We iterate through the community and get the dispatching
    param_tech_no_batt=param_tech.copy()
    param_tech_no_batt.update({'BatteryCapacity':0,'MaxPower':0})
    nested_out={}
    j=0
    k=0
    PV_size_comm=0
    for i in selection.index:
        print(i, end='')
        if selection.loc[i,'sub_'+str(out_comm_final['pv_penetration'])+'_100']:#all with PV
            PV_size_comm+=selection.PV_size[i]
            if selection.loc[i,'sub_'+str(out_comm_final['pv_penetration'])+'_'+str(out_comm_final['batt_penetration'])]: #if battery
                nested_out[i]=dispatch_probs(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech)
                j+=1
            else: #if only PV battery=0 kWh
                nested_out[i]=dispatch_probs(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
                k+=1
        else: #No PV
            nested_out[i]=dispatch_probs(df_CH.E_PV*0,df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
    #get all the data in a single dict
    pv2inv=pd.DataFrame()
    res_pv=pd.DataFrame()
    pv2store=pd.DataFrame()
    inv2load=pd.DataFrame()
    grid2load=pd.DataFrame()
    store2inv=pd.DataFrame()
    LevelOfCharge=pd.DataFrame()
    inv2grid=pd.DataFrame()
    batt_losses=pd.DataFrame()
    inv_losses=pd.DataFrame()
    store2grid=pd.DataFrame()
    flag_sp_p2p=pd.DataFrame()
    flag_no_sp_p2p=pd.DataFrame()
    for i in nested_out.keys():
        pv2inv['pv2inv'+str(i)]=nested_out[i]['pv2inv']
        pv2store['pv2store'+str(i)]=nested_out[i]['pv2store']
        inv2load['inv2load'+str(i)]=nested_out[i]['inv2load']
        store2inv['store2inv'+str(i)]=nested_out[i]['store2inv']
        store2grid['store2grid'+str(i)]=nested_out[i]['store2grid']
        LevelOfCharge['LevelOfCharge'+str(i)]=nested_out[i]['LevelOfCharge']
        batt_losses['batt_losses'+str(i)]=nested_out[i]['batt_losses']
        inv_losses['inv_losses'+str(i)]=nested_out[i]['inv_losses']
        flag_no_sp_p2p['flag_no_sp_p2p'+str(i)]=nested_out[i]['flag_no_sp_p2p']
        flag_sp_p2p['flag_sp_p2p'+str(i)]=nested_out[i]['flag_sp_p2p']
        res_pv['res_pv'+str(i)]=nested_out[i]['res_pv']#not important, recalculated later
        inv2grid['inv2grid'+str(i)]=nested_out[i]['inv2grid']#not important, recalculated later
        grid2load['grid2load'+str(i)]=nested_out[i]['grid2load']#not important, recalculated later

    nested_dict={'pv2inv':pv2inv.sum(axis=1),'res_pv':res_pv.sum(axis=1),'pv2store':pv2store.sum(axis=1),
                 'inv2load':inv2load.sum(axis=1),'grid2load':grid2load.sum(axis=1),
                 'store2inv':store2inv.sum(axis=1),'LevelOfCharge':LevelOfCharge.sum(axis=1),
                 'inv2grid':inv2grid.sum(axis=1),'batt_losses':batt_losses.sum(axis=1),
                 'inv_losses':inv_losses.sum(axis=1),'flag_no_sp_p2p':flag_no_sp_p2p.sum(axis=1),
                 'flag_sp_p2p':flag_sp_p2p.sum(axis=1),'store2grid':store2grid.sum(axis=1)}
    param_tech_no_batt.update({'InverterEfficiency':1})#nested_dict['inv2grid'] is PV generation already in AC used as input for next step
    out_comm_res=dispatch_probs(nested_dict['inv2grid'],nested_dict['grid2load'],param_tech_no_batt)
    out_comm_final={}
    out_comm_final['pv2inv']=nested_dict['pv2inv']
    out_comm_final['pv2store']=nested_dict['pv2store']
    out_comm_final['inv2load']=(nested_dict['inv2load']+out_comm_res['inv2load'])
    out_comm_final['store2inv']=nested_dict['store2inv']
    out_comm_final['LevelOfCharge']=nested_dict['LevelOfCharge']
    out_comm_final['res_pv']=out_comm_res['res_pv']
    out_comm_final['inv2grid']=out_comm_res['inv2grid']
    out_comm_final['grid2load']=out_comm_res['grid2load']
    out_comm_final['batt_losses']=nested_dict['batt_losses']
    out_comm_final['inv_losses']=nested_dict['inv_losses']

    out_comm_final['flag_no_sp_p2p']=nested_dict['flag_no_sp_p2p']
    out_comm_final['flag_sp_p2p']=nested_dict['flag_sp_p2p']
    print(out_comm_final['flag_sp_p2p'].sum())
    out_comm_final['store2grid']=nested_dict['store2grid']
    out_comm_final['PV_size_comm']=PV_size_comm
    out_comm_final['df']=df_CH
    out_comm_final['param_tech']=param_tech
    out_comm_final['param_tech_no_batt']=param_tech_no_batt
    out_comm_final['nested_dict']=nested_dict
    out_comm_final['pv_penetration']=pv_penetration
    out_comm_final['batt_penetration']=batt_penetration
    out_comm_final['selection']=selection



    list_names=[str(i) for i in selection.iloc[:,1]]

    if save:
        comm.save_obj(out_comm_final,'community_p2p'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
    comm.print_analysis(df_CH.E_PV*PV_size_comm, df_CH.loc[:,list_names].sum(axis=1),param_tech_no_batt,
                   out_comm_final,save=True)
    out_comm_final['flag_no_sp_p2p'].sum()


In [10]:
@fn_timer
def run_comm_p2p(inputs):
    '''
    Description
    -----------
    Core function of the P2P community. Does the dispatch of electricity for every
    household taking into account whether the houses have or not PV and battery for the selected 
    PV and battery penetration parameters as well as the probabilities to export to the grid.
    Recalculates the outputs for the whole community taking the surplus generated inside the 
    community (new input as PV generation for the dispatching) and the residual load after 
    self-consumption (new input as demand for the dispatching). Saves the results in pickle objects 
    to calculate the bill after.
    
    Parameters
    -----------
    inputs (array): [use_cluster1_probabilities,save]
                    use_cluster1_probabilities (Boolean): Whether to use cluster 1 probs or not (cluster 2)
                    save (Boolean): Whether to save or not the community_p2p object (dict)

    Returns
    -----------
    None
    Comments
    -----------
    if prob_choice len is one then the prices are defined using 50% of battery SOC all day round. 
    If prob_choice is 3 then the three choices are used to create the supply at morning, midday and evening.
    '''
    community_size=100
    timestep=1 # in hours
    use_cluster1_probabilities=inputs[0]#cluster1 is Price+Autarky cluster2 Mostly Autarkie
    save=inputs[1]
    list_pv_penetration=[100,75,50,25]# %
    list_batt_penetration=[100,75,50,25]# %
    path=os.getcwd()+'/Switzerland/'
    #define the matrix of probs
    # while surplus
    
    if use_cluster1_probabilities:
        prob_mat2=pd.read_csv(path+'Input/surplus_cluster1_p_selling.csv',sep=';',engine='python',index_col=[0])
        #2h before surplus
        prob_mat1=pd.read_csv(path+'Input/nosurplus_cluster1_p_selling.csv',sep=';',engine='python',index_col=[0])
        #14h before surplus
        prob_mat3=pd.read_csv(path+'Input/nosurplus_cluster1_p_selling.csv',sep=';',engine='python',index_col=[0])
    else:#use cluster 2
        prob_mat2=pd.read_csv(path+'Input/surplus_cluster2_p_selling.csv',sep=';',engine='python',index_col=[0])
        #2h before surplus
        prob_mat1=pd.read_csv(path+'Input/nosurplus_cluster2_p_selling.csv',sep=';',engine='python',index_col=[0])
        #14h before surplus
        prob_mat3=pd.read_csv(path+'Input/nosurplus_cluster2_p_selling.csv',sep=';',engine='python',index_col=[0])
    
    for pv_penetration in list_pv_penetration:
        for batt_penetration in list_batt_penetration:
            print('\n########################################')
            print('PV penetration {}'.format(pv_penetration))
            print('Battery penetration {}'.format(batt_penetration))
            out_comm_final=comm.load_obj('community_as_today'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
            param_tech=out_comm_final['param_tech']

            df_CH=out_comm_final['df']
            selection=out_comm_final['selection']

            param_tech.update({'kW_dis':1,'PV_penetration':pv_penetration/100,'Batt_penetration':batt_penetration/100,'prob_choice':[0.35,0.5,.95],
                        'prices':np.array([0, 0.07,.10,.13,.16,.19,.22,.25,.28]),'community_size':community_size,
                        'prob_mat1':prob_mat1,'prob_mat2':prob_mat2,'prob_mat3':prob_mat3,'use_cluster1_probabilities':use_cluster1_probabilities,
                        'interval_SOC':np.array([0,0.05,0.2,0.35,0.5,0.65,0.8,0.95,1.0001]),'path':path})
            price_probability(df_CH,selection,param_tech,pv_penetration,batt_penetration)

            # We set the battery capacity to zero in some cases to get the direct SC only
            # We iterate through the community and get the dispatching
            param_tech_no_batt=param_tech.copy()
            param_tech_no_batt.update({'BatteryCapacity':0,'MaxPower':0})
            nested_out={}
            j=0
            k=0
            PV_size_comm=0
            for i in selection.index:
                print(i, end='')
                if selection.loc[i,'sub_'+str(out_comm_final['pv_penetration'])+'_100']:#all with PV
                    PV_size_comm+=selection.PV_size[i]
                    if selection.loc[i,'sub_'+str(out_comm_final['pv_penetration'])+'_'+str(out_comm_final['batt_penetration'])]: #if battery
                        nested_out[i]=dispatch_probs(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech)
                        j+=1
                    else: #if only PV battery=0 kWh
                        nested_out[i]=dispatch_probs(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
                        k+=1
                else: #No PV
                    nested_out[i]=dispatch_probs(df_CH.E_PV*0,df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
            #get all the data in a single dict
            pv2inv=pd.DataFrame()
            res_pv=pd.DataFrame()
            pv2store=pd.DataFrame()
            inv2load=pd.DataFrame()
            grid2load=pd.DataFrame()
            store2inv=pd.DataFrame()
            LevelOfCharge=pd.DataFrame()
            inv2grid=pd.DataFrame()
            batt_losses=pd.DataFrame()
            inv_losses=pd.DataFrame()
            store2grid=pd.DataFrame()
            flag_sp_p2p=pd.DataFrame()
            flag_no_sp_p2p=pd.DataFrame()
            bill_individual=pd.Series()
            for i in nested_out.keys():
                pv2inv['pv2inv'+str(i)]=nested_out[i]['pv2inv']
                pv2store['pv2store'+str(i)]=nested_out[i]['pv2store']
                inv2load['inv2load'+str(i)]=nested_out[i]['inv2load']
                store2inv['store2inv'+str(i)]=nested_out[i]['store2inv']
                store2grid['store2grid'+str(i)]=nested_out[i]['store2grid']

                LevelOfCharge['LevelOfCharge'+str(i)]=nested_out[i]['LevelOfCharge']
                batt_losses['batt_losses'+str(i)]=nested_out[i]['batt_losses']
                inv_losses['inv_losses'+str(i)]=nested_out[i]['inv_losses']
                flag_no_sp_p2p['flag_no_sp_p2p'+str(i)]=nested_out[i]['flag_no_sp_p2p']
                flag_sp_p2p['flag_sp_p2p'+str(i)]=nested_out[i]['flag_sp_p2p']
                res_pv['res_pv'+str(i)]=nested_out[i]['res_pv']#not important, recalculated later
                inv2grid['inv2grid'+str(i)]=nested_out[i]['inv2grid']#not important, recalculated later
                grid2load['grid2load'+str(i)]=nested_out[i]['grid2load']#not important, recalculated later
                bill_individual.at['bill'+str(i)]=comm.get_bill(nested_out[i],param_tech)
            nested_dict={'pv2inv':pv2inv.sum(axis=1),'res_pv':res_pv.sum(axis=1),'pv2store':pv2store.sum(axis=1),
                         'inv2load':inv2load.sum(axis=1),'grid2load':grid2load.sum(axis=1),
                         'store2inv':store2inv.sum(axis=1),'LevelOfCharge':LevelOfCharge.sum(axis=1),
                         'inv2grid':inv2grid.sum(axis=1),'batt_losses':batt_losses.sum(axis=1),
                         'inv_losses':inv_losses.sum(axis=1),'flag_no_sp_p2p':flag_no_sp_p2p.sum(axis=1),
                         'flag_sp_p2p':flag_sp_p2p.sum(axis=1),'store2grid':store2grid.sum(axis=1)}
            param_tech_no_batt.update({'InverterEfficiency':1})#nested_dict['inv2grid'] is PV generation already in AC used as input for next step
            out_comm_res=dispatch_probs(nested_dict['inv2grid'],nested_dict['grid2load'],param_tech_no_batt)
            out_comm_final={}
            out_comm_final['pv2inv']=nested_dict['pv2inv']
            out_comm_final['pv2store']=nested_dict['pv2store']
            out_comm_final['inv2load']=(nested_dict['inv2load']+out_comm_res['inv2load'])
            out_comm_final['store2inv']=nested_dict['store2inv']
            out_comm_final['LevelOfCharge']=nested_dict['LevelOfCharge']
            out_comm_final['res_pv']=out_comm_res['res_pv']
            out_comm_final['inv2grid']=out_comm_res['inv2grid']
            out_comm_final['grid2load']=out_comm_res['grid2load']
            out_comm_final['batt_losses']=nested_dict['batt_losses']
            out_comm_final['inv_losses']=nested_dict['inv_losses']
            
            
            
            out_comm_final['flag_no_sp_p2p']=nested_dict['flag_no_sp_p2p']
            out_comm_final['flag_sp_p2p']=nested_dict['flag_sp_p2p']
            out_comm_final['store2grid']=nested_dict['store2grid']
            out_comm_final['PV_size_comm']=PV_size_comm
            out_comm_final['df']=df_CH
            out_comm_final['param_tech']=param_tech
            out_comm_final['param_tech_no_batt']=param_tech_no_batt
            out_comm_final['nested_dict']=nested_dict
            out_comm_final['pv_penetration']=pv_penetration
            out_comm_final['batt_penetration']=batt_penetration
            out_comm_final['selection']=selection
            bill_individual.at['pv_penetration']=pv_penetration
            bill_individual.at['batt_penetration']=batt_penetration

            list_names=[str(i) for i in selection.iloc[:,1]]

            if save:
                if use_cluster1_probabilities:
                    comm.save_obj(out_comm_final,'community_p2p_c1'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
                    comm.save_obj(inv2grid,'i2g_community_p2p_c1'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
                    comm.save_obj(grid2load,'g2l_community_p2p_c1'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
                else:
                    comm.save_obj(out_comm_final,'community_p2p_c2'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
                    comm.save_obj(inv2grid,'i2g_community_p2p_c2'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
                    comm.save_obj(grid2load,'g2l_community_p2p_c2'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
            comm.print_analysis(df_CH.E_PV*PV_size_comm, df_CH.loc[:,list_names].sum(axis=1),param_tech_no_batt,
                           out_comm_final,save=True)
            comm.save_results(bill_individual,'aggregated_results_bill_p2p.csv')
            print(out_comm_final['flag_sp_p2p'].sum())
            print('out of iteration')
            print('################')

In [12]:
%%time
if __name__ == '__main__':
    inputs=[False,True]
    run_comm_p2p(inputs)


########################################
PV penetration 100
Battery penetration 100
0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899
%%%%%%%%%%%%%%%%%
Printing analysis
inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 228428 kWh
Direct Self Consumption: 109395 kWh
Indirect Self Consumption: 119033 kWh
Total fed to the grid from PV: 294255 kWh
Total fed to the grid from battery: 100058 kWh
The amount of times energy when surplus was exchanged was: 85665.0
The amount of times energy was exchanged when no surplus was: 122885.0
Total fed to the grid: 394313 kWh
Total bought from the grid: 33680.5 kWh
Self consumption rate (SCR): 33.3778%
Self sufficiency rate (SSR): 87.1502%
Self consumption rate (SCR): 33.4%
Self sufficiency rate (SSR): 87.2%
Amount of energy provided by the battery: 219091 kWh
Ave

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899
%%%%%%%%%%%%%%%%%
Printing analysis
inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 220745 kWh
Direct Self Consumption: 130737 kWh
Indirect Self Consumption: 90008.5 kWh
Total fed to the grid from PV: 332346 kWh
Total fed to the grid from battery: 74742.4 kWh
The amount of times energy when surplus was exchanged was: 63987.0
The amount of times energy was exchanged when no surplus was: 92144.0
Total fed to the grid: 407088 kWh
Total bought from the grid: 41363.6 kWh
Self consumption rate (SCR): 32.2552%
Self sufficiency rate (SSR): 84.2189%
Self consumption rate (SCR): 32.3%
Self sufficiency rate (SSR): 84.2%
Amount of energy provided by the battery: 164751 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899
%%%%%%%%%%%%%%%%%
Printing analysis
inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 204385 kWh
Direct Self Consumption: 144253 kWh
Indirect Self Consumption: 60131.9 kWh
Total fed to the grid from PV: 373835 kWh
Total fed to the grid from battery: 54345.3 kWh
The amount of times energy when surplus was exchanged was: 46516.0
The amount of times energy was exchanged when no surplus was: 62397.0
Total fed to the grid: 428181 kWh
Total bought from the grid: 57724.1 kWh
Self consumption rate (SCR): 29.8646%
Self sufficiency rate (SSR): 77.977%
Self consumption rate (SCR): 29.9%
Self sufficiency rate (SSR): 78%
Amount of energy provided by the battery: 114477 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
To

...

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899
%%%%%%%%%%%%%%%%%
Printing analysis
inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 162585 kWh
Self Consumption: 108783 kWh
Direct Self Consumption: 97011.5 kWh
Indirect Self Consumption: 11771.1 kWh
Total fed to the grid from PV: 25132.4 kWh
Total fed to the grid from battery: 16287.2 kWh
The amount of times energy when surplus was exchanged was: 13948.0
The amount of times energy was exchanged when no surplus was: 18134.0
Total fed to the grid: 41419.6 kWh
Total bought from the grid: 153326 kWh
Self consumption rate (SCR): 66.908%
Self sufficiency rate (SSR): 41.5028%
Self consumption rate (SCR): 66.9%
Self sufficiency rate (SSR): 41.5%
Amount of energy provided by the battery: 28058.3 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: in

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899
%%%%%%%%%%%%%%%%%
Printing analysis
inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 162585 kWh
Self Consumption: 98501.2 kWh
Direct Self Consumption: 93613.6 kWh
Indirect Self Consumption: 4887.63 kWh
Total fed to the grid from PV: 44593.4 kWh
Total fed to the grid from battery: 8478.35 kWh
The amount of times energy when surplus was exchanged was: 7256.0
The amount of times energy was exchanged when no surplus was: 10121.0
Total fed to the grid: 53071.8 kWh
Total bought from the grid: 163608 kWh
Self consumption rate (SCR): 60.5843%
Self sufficiency rate (SSR): 37.5803%
Self consumption rate (SCR): 60.6%
Self sufficiency rate (SSR): 37.6%
Amount of energy provided by the battery: 13366 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


In [ ]:
%%time
if __name__ == '__main__':
    # start "number of cores" processes
    print('welcome to community p2p')
    inputs_list=[[False,False],[True,False]]# use_cluster1_probabilities=inputs[0]#cluster1 is Price+Autarky cluster2 Mostly Autarkie
                            #save=inputs[1]
    for inputs in inputs_list:
        
        mp.freeze_support()
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
        pool=mp.Pool(10)

        

        pool.map(run_comm_p2p,np.tile(inputs,(100,1)))
        pool.close()
        pool.join()

welcome to community p2p
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

########################################
########################################
########################################
########################################
########################################
########################################
########################################

########################################
########################################
########################################





PV penetration 100


PV penetration 100

PV penetration 100
PV penetration 100
PV penetration 100
PV penetration 100
PV penetration 100
PV penetration 100
Battery penetration 100
PV penetration 100
Battery penetration 100
Battery penetration 100
PV penetration 100
Battery penetration 100
Battery penetration 100
Battery penetration 100
Battery penetration 100
Battery penetration 100
Battery penetration 100
Battery penetration 100
012345678910111213141516171819202122232425262728293031323334

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 228374 kWh
Direct Self Consumption: 109274 kWh
Indirect Self Consumption: 119100 kWh
Total fed to the grid from PV: 294495 kWh
Total fed to the grid from battery: 99884.6 kWh
The amount of times energy when surplus was exchanged was: 85497.0
The amount of times energy was exchanged when no surplus was: 122866.0
Total fed to the grid: 394380 kWh
Total bought from the grid: 33734.8 kWh
Self consumption rate (SCR): 33.3699%
Self sufficiency rate (SSR): 87.1295%
Self consumption rate (SCR): 33.4%
Self sufficiency rate (SSR): 87.1%
Amount of energy provided by the battery: 218984 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 21676.6 kWh
Total inverter losses: 39750.3 kWh
Total bill: -11724 USD/a 
Residue (check): -1.52966e-10 kWh
%%%%%%%%%%%%%%%%%
True
%%%%%%%%%%%%%%%%%
Saving analysis
85497.0
out of iteration
###############

comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 228374 kWh
Direct Self Consumption: 109274 kWh
Indirect Self Consumption: 119100 kWh
Total fed to the grid from PV: 294495 kWh
Total fed to the grid from battery: 99884.6 kWh
The amount of times energy when surplus was exchanged was: 85497.0
The amount of times energy was exchanged when no surplus was: 122866.0
Total fed to the grid: 394380 kWh
Total bought from the grid: 33734.8 kWh
Self consumption rate (SCR): 33.3699%
Self sufficiency rate (SSR): 87.1295%
Self consumption rate (SCR): 33.4%
Self sufficiency rate (SSR): 87.1%
Amount of energy provided by the battery: 218984 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 21676.6 kWh
Total inverter losses: 39750.3 kWh
Total bill: -11724 USD/a 
Residue (check): -1.52966e-10 kWh
%%%%%%%%%%%%%%%%%
True
%%%%%%%%%%%%%%%%%
Saving analysis
85497.0
out of iteration
###############

...

inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 220738 kWh
Direct Self Consumption: 130812 kWh
Indirect Self Consumption: 89925.4 kWh
Total fed to the grid from PV: 332181 kWh
Total fed to the grid from battery: 74919.9 kWh
The amount of times energy when surplus was exchanged was: 64137.0
The amount of times energy was exchanged when no surplus was: 92014.0
Total fed to the grid: 407101 kWh
Total bought from the grid: 41371 kWh
Self consumption rate (SCR): 32.2541%
Self sufficiency rate (SSR): 84.2161%
Self consumption rate (SCR): 32.3%
Self sufficiency rate (SSR): 84.2%
Amount of energy provided by the battery: 164845 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 16317.3 kWh
Total inverter losses: 40074.8 kWh
Total bill: -10926 USD/a 
Residue (check): -7.48628e-11 kWh
%%%%%%%%%%%%%%%%%
True
%%%%%%%%%%%%%%%%%
Saving analysis
64137.0
out of iteration
################


comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 220810 kWh
Direct Self Consumption: 130801 kWh
Indirect Self Consumption: 90008.7 kWh
Total fed to the grid from PV: 332109 kWh
Total fed to the grid from battery: 74908.2 kWh
The amount of times energy when surplus was exchanged was: 64126.0
The amount of times energy was exchanged when no surplus was: 91960.0
Total fed to the grid: 407017 kWh
Total bought from the grid: 41299 kWh
Self consumption rate (SCR): 32.2646%
Self sufficiency rate (SSR): 84.2436%
Self consumption rate (SCR): 32.3%
Self sufficiency rate (SSR): 84.2%
Amount of energy provided by the battery: 164917 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 16324.9 kWh
Total inverter losses: 40074 kWh
Total bill: -10920.1 USD/a 
Residue (check): -6.45173e-11 kWh
%%%%%%%%%%%%%%%%%
True
%%%%%%%%%%%%%%%%%
Saving analysis
64126.0
out of iteration
################


comm_as_today_run.ipynb:34: RuntimeWarning: divide by zero encountered in double_scalars
  "metadata": {},


inside st2gr
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 220734 kWh
Direct Self Consumption: 130725 kWh
Indirect Self Consumption: 90009.4 kWh
Total fed to the grid from PV: 332336 kWh
Total fed to the grid from battery: 74768.1 kWh
The amount of times energy when surplus was exchanged was: 63999.0
The amount of times energy was exchanged when no surplus was: 92214.0
Total fed to the grid: 407104 kWh
Total bought from the grid: 41374.4 kWh
Self consumption rate (SCR): 32.2536%
Self sufficiency rate (SSR): 84.2148%
Self consumption rate (SCR): 32.3%
Self sufficiency rate (SSR): 84.2%
Amount of energy provided by the battery: 164777 kWh
Average Charging/Discharging depth: inf
Number of equivalent full cycles per year: inf 
Total battery losses: 16311.2 kWh
Total inverter losses: 40074.8 kWh
Total bill: -10919.2 USD/a 
Residue (check): -1.00442e-10 kWh
%%%%%%%%%%%%%%%%%
True
%%%%%%%%%%%%%%%%%
Saving analysis
63999.0
out of iteration
#############